In [4]:
import logging
import os
import sys
import json
from typing import List

from langchain.agents.agent import AgentExecutor
from langchain_core.tools import Tool, StructuredTool
from langchain_aws.agents.bedrock.agent_base import agent_tool, BedrockAgentMetadata, BedrockAgentBase
from langchain_aws.agents.bedrock.agent_executor import BedrockAgentExecutor
from langchain_aws.agents.bedrock.bedrock_agent import BedrockAgent

logging.basicConfig(format='%(message)s', level=logging.INFO)

In [5]:

agent_name: str = "MortgageEvaluatorAgent_1"
agent_instruction: str = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"
agent_region: str = "us-east-1"

@agent_tool(action_group='AssetDetail', action='getAssetValue')
def getAssetValue(
        asset_holder_id: str = ' '
) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: id of the owner holding the asset
    Returns:
        str -> the valuation of the asset
    """
    return f"The total asset value for {asset_holder_id} is 100K"


@agent_tool
def getMortgageRate(
        asset_holder_id: str = ' ',
        asset_value: str = ' '
) -> str:
    """
    Get the mortage rate based on asset value

    Args:
        asset_holder_id: id of the owner holding the asset
        asset_value: asset value which is used to get the mortgage rate
    Returns:
        str -> the calculated mortage rate based on the asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"

agent_tools: List[Tool] = [getAssetValue, getMortgageRate]

def handle_trace(event=dict):
    logging.info("\n********************** Event **********************\n")
    logging.info(f"{json.dumps(event, indent=4)} \n")

In [6]:
bedrock_agent = BedrockAgent(
    agent_name=agent_name,
    agent_instruction=agent_instruction,
    agent_region=agent_region,
    agent_tools=agent_tools,
    trace_handler=handle_trace
)

agent_executor = BedrockAgentExecutor.from_agent_and_tools(
    agent=bedrock_agent,
    verbose=True,
    tools=agent_tools,
    return_intermediate_steps=True,
    max_iterations=8
)

Found credentials in shared credentials file: ~/.aws/credentials
Created AWS Bedrock agent with request id: 462ac4a0-f9fc-46ef-a69f-f305b869c120
Created Bedrock agent ActionGroup AssetDetail and Function with request id: 48ce6769-25a7-4dab-a8a4-7792f3b01f50
Created Bedrock agent ActionGroup getMortgageRateActionGroup and Function with request id: ba5898ca-0545-4623-a0d6-dfedf73d1cea
Prepared AWS Bedrock agent with request id: e35e2008-160c-4fd8-b6a1-015c61bf51fb


In [7]:
agent_executor.invoke({"input": "what is my mortgage rate for id AVC-1234"})



> Entering new BedrockAgentExecutor chain...


Bedrock agent call request id: 4fff235b-cde7-42d1-8058-e1e0c0b111ad

********************** Bedrock Agents Input **********************

Input: what is my mortgage rate for id AVC-1234 

SessionState: {} 


********************** Event **********************

{
    "rationale": {
        "text": "To get the mortgage rate for the asset holder with id AVC-1234, I first need to get the asset value for that id using the AssetDetail::getAssetValue function. Then I can pass that asset value to the getMortgageRateActionGroup::getMortgageRate function to get the mortgage rate.",
        "traceId": "4fff235b-cde7-42d1-8058-e1e0c0b111ad-0"
    }
} 


********************** Event **********************

{
    "invocationInput": {
        "actionGroupInvocationInput": {
            "actionGroupName": "AssetDetail",
            "function": "getAssetValue",
            "parameters": [
                {
                    "name": "asset_holder_id",
                    "type": "string",
             


> Finished chain.


{'input': 'what is my mortgage rate for id AVC-1234',
 'output': 'The mortgage rate for id AVC-1234 is 8.87%'}

In [5]:
bedrock_agent.delete()

Deleted Bedrock agent call request id: 7359a787-7c38-43cd-b260-a47407c3eb9c
